# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
stored_data_file = "../output_data/cities.csv"
# output_data_file

weather_df = pd.read_csv(stored_data_file)

weather_df

,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Medvezh'yegorsk,52,RU,90,62.9167,34.4667,-11.38,2.15
1,Nemuro,75,JP,86,43.3236,145.5750,17.60,5.75
2,Jamestown,90,US,73,42.0970,-79.2353,19.99,5.75
3,Dikson,29,RU,88,73.5069,80.5464,-21.62,7.94
4,Meulaboh,87,ID,89,4.1363,96.1285,76.51,4.18
...,...,...,...,...,...,...,...,...
577,Makat,99,KZ,96,47.6469,53.3463,32.79,2.44
578,Muş Province,57,TR,96,39.0000,41.7500,22.91,2.33
579,Sabang,53,ID,74,5.8933,95.3214,81.68,5.73
580,Leningradskiy,11,RU,83,69.3833,178.4167,-28.14,7.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

In [15]:
locations = weather_df[["Latitude", "Longitude"]]
# locations

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
new_weather_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_weather_df["Max Temp"] <= 80) & (new_weather_df["Max Temp"] > 70)
wind_speed = new_weather_df["Wind Speed"] < 10
cloudiness = new_weather_df["Cloudiness"] == 0

new_weather_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
9,Saint-Philippe,77.00,5.75,0
75,Saint-Pierre,77.00,5.75,0
96,La Reforma,77.00,3.00,0
99,Hobart,72.00,9.22,0
171,Cabo San Lucas,75.00,1.99,0
236,Ancud,75.20,5.75,0
364,Doka,73.09,8.70,0
366,Porto Seguro,77.00,8.05,0
530,Salalah,73.40,8.05,0
532,Ouargaye,76.57,8.25,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = pd.DataFrame(weather_df, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Medvezh'yegorsk,RU,62.9167,34.4667,
1,Nemuro,JP,43.3236,145.5750,
2,Jamestown,US,42.0970,-79.2353,
3,Dikson,RU,73.5069,80.5464,
4,Meulaboh,ID,4.1363,96.1285,
...,...,...,...,...,...
577,Makat,KZ,47.6469,53.3463,
578,Muş Province,TR,39.0000,41.7500,
579,Sabang,ID,5.8933,95.3214,
580,Leningradskiy,RU,69.3833,178.4167,


In [23]:
target_coordinates = f"{hotel_df['Latitude'][0]},{hotel_df['Longitude'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [28]:
print(response.json())

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 62.9056598, 'lng': 34.4632463}, 'viewport': {'northeast': {'lat': 62.90721777989271, 'lng': 34.46429047989272}, 'southwest': {'lat': 62.90451812010727, 'lng': 34.46159082010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Гостиница «Прованс»', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2250, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112986779325686722456">Сергей Ф</a>'], 'photo_reference': 'ATtYBwJTqHtAD1vEYkcnGBCrGhEl2y9IOtVCDNlH2tQSVH7o2tOpfcOPiohOjpZqx8XWU033RtwDuAY2dHggyFMzI5uiWlUYzXO6BtTZTUmTjV1a75MNWkZBeYXgkKJh4G4n9LQzzbIe-gTfjdsqPwR8Bqj7t2NDu1bRvHKcedEOiV8fh-zR', 'width': 4000}], 'place_id': 'ChIJ6YvZ7W_Cn0YRV26HAH6vkcg', 'plus_code': {'compound_code': 'WF47+77 Medvezhyegorsk, Republic of Karelia, Russia', 'global_code': '9GJPWF47+77'}, 'rating': 4.2, 'reference': 'ChIJ6YvZ7W_Cn0

In [29]:
hotel_df.iloc[0][0]

"Medvezh'yegorsk"

In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))